In [ ]:
sales_analysis.ipynb


In [1]:
import pandas as pd
import numpy as np
print("tools imported sucess!")

tools imported sucess!


In [2]:
df=pd.read_csv('sales_data.csv')
print("data upload sucess!")
print(f"数据集形状：{df.shape}")

data upload sucess!
数据集形状：(8, 7)


In [3]:
df.head()

订单ID  产品名称  数量    单价        订单日期 客户类型  城市
0  1001  苹果手机   1  5999  2024-01-15  新客户  北京
1  1002  华为耳机   2   399  2024-01-15  老客户  上海
2  1003  小米手环   3   199  2024-01-16  新客户  广州
3  1004  苹果手机   1  5999  2024-01-17  老客户  北京
4  1005  三星平板   1  3299  2024-01-18  新客户  深圳

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   订单ID    8 non-null      int64 
 1   产品名称    8 non-null      object
 2   数量      8 non-null      int64 
 3   单价      8 non-null      int64 
 4   订单日期    8 non-null      object
 5   客户类型    8 non-null      object
 6   城市      8 non-null      object
dtypes: int64(3), object(4)
memory usage: 580.0+ bytes


In [5]:
df.describe()

订单ID        数量           单价
count     8.00000  8.000000     8.000000
mean   1004.50000  2.000000  2811.500000
std       2.44949  1.414214  2828.648087
min    1001.00000  1.000000   199.000000
25%    1002.75000  1.000000   349.000000
50%    1004.50000  1.500000  1849.000000
75%    1006.25000  2.250000  5999.000000
max    1008.00000  5.000000  5999.000000

In [6]:
# 创建销售额字段
df['销售额'] = df['数量'] * df['单价']

# 检查数据
print("新增销售额字段后数据预览:")
print(df.head())

print(f"\n总销售额: ¥{df['销售额'].sum():,.2f}")

新增销售额字段后数据预览:
   订单ID  产品名称  数量    单价        订单日期 客户类型  城市   销售额
0  1001  苹果手机   1  5999  2024-01-15  新客户  北京  5999
1  1002  华为耳机   2   399  2024-01-15  老客户  上海   798
2  1003  小米手环   3   199  2024-01-16  新客户  广州   597
3  1004  苹果手机   1  5999  2024-01-17  老客户  北京  5999
4  1005  三星平板   1  3299  2024-01-18  新客户  深圳  3299

总销售额: ¥30,084.00


In [7]:
df.head()

订单ID  产品名称  数量    单价        订单日期 客户类型  城市   销售额
0  1001  苹果手机   1  5999  2024-01-15  新客户  北京  5999
1  1002  华为耳机   2   399  2024-01-15  老客户  上海   798
2  1003  小米手环   3   199  2024-01-16  新客户  广州   597
3  1004  苹果手机   1  5999  2024-01-17  老客户  北京  5999
4  1005  三星平板   1  3299  2024-01-18  新客户  深圳  3299

In [8]:
df.shape()

TypeError: 'tuple' object is not callable

In [9]:
df.shape

(8, 8)

In [10]:
new_customer_count = df[df['客户类型'] == '新客户'].groupby('城市')['订单ID'].nunique()
print("各城市新客户数量：")
print(new_customer_count)

各城市新客户数量：
城市
北京    1
广州    2
深圳    1
Name: 订单ID, dtype: int64


In [11]:
city_metrics = df.groupby('城市').agg({
    '销售额': 'sum',
    '订单ID': 'nunique'
}).rename(columns={'订单ID': '总订单数'})

city_metrics['平均客单价'] = city_metrics['销售额'] / city_metrics['总订单数']
print("\n各城市平均客单价：")
print(city_metrics[['平均客单价']])


各城市平均客单价：
          平均客单价
城市             
上海   598.500000
北京  7998.666667
广州   796.000000
深圳  3299.000000


In [12]:
# 创建开店优先级数据
priority_df = pd.DataFrame({
    '新客户数量': new_customer_count,
    '平均客单价': city_metrics['平均客单价']
})

# 计算开店优先级 = 新客户数量 × 平均客单价
priority_df['开店优先级'] = priority_df['新客户数量'] * priority_df['平均客单价']

# 按优先级排序
priority_df = priority_df.sort_values('开店优先级', ascending=False)

print("=== 开店优先级分析 ===")
print(priority_df)

print(f"\n🎯 推荐开店城市：{priority_df.index[0]}")
print(f"   - 新客户数量：{priority_df.iloc[0]['新客户数量']}个")
print(f"   - 平均客单价：¥{priority_df.iloc[0]['平均客单价']:,.0f}")
print(f"   - 优先级分数：{priority_df.iloc[0]['开店优先级']:.0f}")

=== 开店优先级分析 ===
    新客户数量        平均客单价        开店优先级
城市                                 
北京    1.0  7998.666667  7998.666667
深圳    1.0  3299.000000  3299.000000
广州    2.0   796.000000  1592.000000
上海    NaN   598.500000          NaN

🎯 推荐开店城市：北京
   - 新客户数量：1.0个
   - 平均客单价：¥7,999
   - 优先级分数：7999
